# Xarray

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

np.random.seed(1001)
da = xr.DataArray(
    np.random.randint(low = 1, high = 4, size = (3, 2, 4)),
    [
        ("time", pd.date_range("2000-01-01", periods=3)),
        ("space", ["IA", "IL"]),
        ("number", range(4))
    ],
)
da

<xarray.DataArray (time: 3, space: 2, number: 4)>
array([[[2, 2, 2, 3],
        [3, 1, 1, 3]],

       [[2, 1, 1, 2],
        [2, 3, 2, 2]],

       [[1, 3, 1, 1],
        [2, 2, 1, 3]]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
  * space    (space) <U2 'IA' 'IL'
  * number   (number) int64 0 1 2 3

## 定位数据(Index selection)及相关操作
| 维度查找 | Index查找 | `DataArray`类操作 | `Dataset`类操作 |
| --- | --- | --- | --- |
| 通过位置 | 通过Index的整数坐标 | `da[:, 0]`遵循多维数组切分查找规则即可 | 无（要想使用必须通过声明变量名后）<br>`ds['sst'][:, 0]` |
| 通过位置 | 通过label | `da.loc[:, 'IA']` | 无 |
| 通过坐标名称 | 通过Index整数坐标 | `da.isel(space = 0)` or<br>`da[dict(space = 0)]` | `ds.isel(space=0)`or<br>`ds[dict(space=0)]` |
| 通过坐标名称 | 通过label | `da.sel(space='IA')` or<br>`da.loc[dict(space='IA')]` | `ds.sel(space='IA')` or<br>`ds.loc[dict(space='IA')]` |


此时我想将`da`的`number`坐标中指定区域`A`的数值替换成`B`，有如下几种思路，分别对应不同的需求：
1. `A`与`B`存在某种线性函数关系，可以直接从`A`计算出`B`
2. `A`与`B`无关，且`B`的值完全相同，可以通过Python的广播机制一次性赋值全部位置
3. `A`与`B`存在复杂关系，或者完全无关，这时候只能通过`B`一一对应位置赋值给`A`来解决问题

为了能精确查找数据，在`ds`,`da`数据类型下，更推荐`sel(), loc[]`

In [2]:
#1. A与B存在某种线性函数关系，可以直接从A计算出B
da.loc[dict(time = '2000-01-01')] *= -2 
da

<xarray.DataArray (time: 3, space: 2, number: 4)>
array([[[-4, -4, -4, -6],
        [-6, -2, -2, -6]],

       [[ 2,  1,  1,  2],
        [ 2,  3,  2,  2]],

       [[ 1,  3,  1,  1],
        [ 2,  2,  1,  3]]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
  * space    (space) <U2 'IA' 'IL'
  * number   (number) int64 0 1 2 3

In [3]:
#2. A与B无关，且B的值完全相同，可以通过Python的广播机制一次性赋值全部位置
da.loc[dict(time = '2000-01-02')] = 0
da

<xarray.DataArray (time: 3, space: 2, number: 4)>
array([[[-4, -4, -4, -6],
        [-6, -2, -2, -6]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 1,  3,  1,  1],
        [ 2,  2,  1,  3]]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
  * space    (space) <U2 'IA' 'IL'
  * number   (number) int64 0 1 2 3

In [4]:
da.sel(time = '2000-01-01', space = 'IA') = 0

SyntaxError: cannot assign to function call (<ipython-input-4-16775e27bb6e>, line 1)

需要注意到，`ds.sel()`与`ds.loc[]`是有本质区别的,`sel()`，`isel()`是xarray的函数方法，也就是说其作用是通过一个函数来切割相应数据，应该遵循函数的规定，切割的值是通过`return`返回出来，自然不能通过对函数右边设置等号来赋值。

而`.loc[]`使用中括号，显然使用的是python自带的数据切割属性，并不是函数，因此在等号右边赋值是完全符合规范的。
同时这两者都有可赋值的属性，因此`loc[]`适用性更为广泛

In [9]:
temp = da.loc[dict(time = '2000-01-01', space = 'IA')]
temp_sel = da.sel(time = '2000-01-01', space = 'IA')
temp, temp_sel

(<xarray.DataArray (number: 4)>
 array([-4, -4, -4, -6])
 Coordinates:
     time     datetime64[ns] 2000-01-01
     space    <U2 'IA'
   * number   (number) int64 0 1 2 3,
 <xarray.DataArray (number: 4)>
 array([-4, -4, -4, -6])
 Coordinates:
     time     datetime64[ns] 2000-01-01
     space    <U2 'IA'
   * number   (number) int64 0 1 2 3)

In [10]:
#3. A与B存在复杂关系，或者完全无关，这时候只能通过B一一对应位置赋值给A来解决问题
np.random.seed(10002)
data = np.random.randint(low = 99, high = 200, size = (2,4))

# method 1 

da.loc[dict(time = '2000-01-03')] = data
da

<xarray.DataArray (time: 3, space: 2, number: 4)>
array([[[ -4,  -4,  -4,  -6],
        [ -6,  -2,  -2,  -6]],

       [[  0,   0,   0,   0],
        [  0,   0,   0,   0]],

       [[196, 196, 150, 161],
        [109, 194, 161, 115]]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
  * space    (space) <U2 'IA' 'IL'
  * number   (number) int64 0 1 2 3

另外也介绍一种可以对全数据进行替换的方式`copy()`，使用在要产生新数组，不影响原数组数据的情况

In [11]:
new_da = np.random.randint(low = 90, high = 100, size = (3,2,4))

copy_test = da.copy(data = new_da)
copy_test, da

(<xarray.DataArray (time: 3, space: 2, number: 4)>
 array([[[97, 99, 92, 98],
         [92, 91, 93, 96]],
 
        [[97, 93, 97, 91],
         [93, 97, 90, 95]],
 
        [[90, 92, 94, 92],
         [92, 95, 93, 93]]])
 Coordinates:
   * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
   * space    (space) <U2 'IA' 'IL'
   * number   (number) int64 0 1 2 3,
 <xarray.DataArray (time: 3, space: 2, number: 4)>
 array([[[ -4,  -4,  -4,  -6],
         [ -6,  -2,  -2,  -6]],
 
        [[  0,   0,   0,   0],
         [  0,   0,   0,   0]],
 
        [[196, 196, 150, 161],
         [109, 194, 161, 115]]])
 Coordinates:
   * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
   * space    (space) <U2 'IA' 'IL'
   * number   (number) int64 0 1 2 3)

## 修改坐标

In [12]:
da.reindex(time =pd.date_range(start = '1999-12-30', end = '2000-01-05', freq = 'D'), method="pad")
# pd.date_range()

<xarray.DataArray (time: 7, space: 2, number: 4)>
array([[[ nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan]],

       [[ -4.,  -4.,  -4.,  -6.],
        [ -6.,  -2.,  -2.,  -6.]],

       [[  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.]],

       [[196., 196., 150., 161.],
        [109., 194., 161., 115.]],

       [[196., 196., 150., 161.],
        [109., 194., 161., 115.]],

       [[196., 196., 150., 161.],
        [109., 194., 161., 115.]]])
Coordinates:
  * time     (time) datetime64[ns] 1999-12-30 1999-12-31 ... 2000-01-05
  * space    (space) <U2 'IA' 'IL'
  * number   (number) int64 0 1 2 3

## 根据坐标查找数据之——选取固定月份所有数据
由于xarray中没有内置的关于月份选择的函数，我们可以根据数据切片选择的思路，加上`pandas.date_rangte()`和`xarray.cftime_range()`方便的时间维度生成来共同完成对需要的月份的选取，下面就以选取2014～2017年间所有1月份的温度数据为例：

In [5]:
temp = 15 + 8 * np.random.randn(2, 2, 48)
lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]

ds = xr.Dataset(
    {
        "temperature": (["x", "y", "time"], temp)
    },
    coords={
        "lon": (["x", "y"], lon),
        "lat": (["x", "y"], lat),
        "time": pd.date_range("2014-01-01", periods=48, freq='MS'),
        "reference_time": pd.Timestamp("2014-01-01"),
    },
)
ds

<xarray.Dataset>
Dimensions:         (time: 48, x: 2, y: 2)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-01-01 2014-02-01 ... 2017-12-01
    reference_time  datetime64[ns] 2014-01-01
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 15.26 17.61 19.98 ... 20.55 12.63 38.5

In [13]:
# 生成一个在规定时间范围内的所有1月的date time数据
jan_sel = xr.cftime_range(start = '2014-01', end = '2017-01', freq = '12MS')
jan_sel

CFTimeIndex([2014-01-01 00:00:00, 2015-01-01 00:00:00, 2016-01-01 00:00:00,
             2017-01-01 00:00:00],
            dtype='object', length=4, calendar='gregorian', freq='AS-JAN')

In [14]:
ds.sel(time = jan_sel)

<xarray.Dataset>
Dimensions:         (time: 4, x: 2, y: 2)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-01-01 2015-01-01 ... 2017-01-01
    reference_time  datetime64[ns] 2014-01-01
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 15.26 30.66 14.43 ... 15.86 16.25 16.83

## xarray中的数值计算

In [19]:
np.random.seed(1001)
mu_1 = xr.DataArray(
    np.random.randint(low = 1, high = 4, size = (3, 4)),
    [
        ("time", pd.date_range("2000-01-01", periods=3)),
        ("number", range(4))
    ],
)
mu_2 = xr.DataArray(
    np.random.randint(low = 10, high = 11, size = (3, 4)),
    [
        ("time", pd.date_range("2000-01-01", periods=3)),
        ("number", range(4))
    ],
)
mu_1, mu_2

(<xarray.DataArray (time: 3, number: 4)>
 array([[2, 2, 2, 3],
        [3, 1, 1, 3],
        [2, 1, 1, 2]])
 Coordinates:
   * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
   * number   (number) int64 0 1 2 3,
 <xarray.DataArray (time: 3, number: 4)>
 array([[10, 10, 10, 10],
        [10, 10, 10, 10],
        [10, 10, 10, 10]])
 Coordinates:
   * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
   * number   (number) int64 0 1 2 3)

In [20]:
mu_1 + mu_2

<xarray.DataArray (time: 3, number: 4)>
array([[12, 12, 12, 13],
       [13, 11, 11, 13],
       [12, 11, 11, 12]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
  * number   (number) int64 0 1 2 3

In [21]:
mu_1 @ mu_2

<xarray.DataArray ()>
array(230)

In [22]:
mu_1 / mu_2

<xarray.DataArray (time: 3, number: 4)>
array([[0.2, 0.2, 0.2, 0.3],
       [0.3, 0.1, 0.1, 0.3],
       [0.2, 0.1, 0.1, 0.2]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
  * number   (number) int64 0 1 2 3

In [23]:
mu_1 * mu_2

<xarray.DataArray (time: 3, number: 4)>
array([[20, 20, 20, 30],
       [30, 10, 10, 30],
       [20, 10, 10, 20]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03
  * number   (number) int64 0 1 2 3